Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

You may remove `raise NotImplementedError()` line and write your own code.

In [ ]:
NAME = "Shriya Reddy Ponnala"

---

# Assignment 1

## Q1. Task Description

### 1. Data Generation:
   - Generate two normal distributions each of size 30,000 with feature dimension=100. (Hint: Shape(2*30000, 100))
   - You may choose arbitrary mean and standard deviation.
   - Label the first set of feature vectors as "+1" and the second set of feature vectors as either "-1" or "0."

     
![alt text](gaussian_bimodal.png)

In [421]:
# YOUR CODE HERE
import numpy as np
size = 30000
dimension = 100
distribution_1 = np.random.normal(0, 1, (size, dimension))
distribution_2 = np.random.normal(4, 6, (size, dimension))
labels_1 = np.ones(size) 
labels_2 = -np.ones(size) 
distribution = np.concatenate((distribution_1, distribution_2))
labels = np.concatenate((labels_1, labels_2))

In [422]:
input = np.concatenate((distribution.reshape(60000, 100), labels.reshape(-1, 1)), axis=1)

### 2. Data Splitting
- Split the generated data into training, validation and test sets in 8:1:1 ratio.
- You are expected to write the code from scratch. (i.e. Do not use train_test_split function from sklearn)

In [423]:
# YOUR CODE HERE
np.random.shuffle(input)
train_info = input[:48000,:100]
train_label = input[:48000,100:]
validation_info = input[48000:54000,:100]
label_validation = input[48000:54000,100:]
test_info = input[54000:,:100]
test_label = input[54000:,100:]

### 3. Classifier Implementation
- Implement Linear Classifiers using three different methods. (i.e. Write code from scratch. Refer lecture slides)
  - Random Method
  - Perceptron Method
  - Gradient Descent

In [424]:
# YOUR CODE HERE
def implementRandomMethod(input,label,sample_size):
    coefficient = np.random.random((sample_size,100))
    coefficient_bias = np.random.random((sample_size,1))
    expected_labels = np.matmul(coefficient,input.T) + coefficient_bias
    for i in range(expected_labels.shape[1]):
        for j in range(expected_labels.shape[0]):
              if(expected_labels[j][i] >= 0):
                expected_labels[j][i] = 1
              else:
                expected_labels[j][i] = -1
        random_best_coefficient = []
        random_best_coefficient_bias = []
        error = input.shape[0]
        for i in range(sample_size):
            temp_error = (expected_labels[i].reshape(-1,1) == label)
            temp_error = np.count_nonzero(temp_error == False)
            if(temp_error < error):
                random_best_coefficient = coefficient[i]
                random_best_coefficient_bias = coefficient_bias[i]
    return random_best_coefficient.reshape(-1,1),random_best_coefficient_bias.reshape(-1,1)

In [425]:
def implementRandomMethodPredict(input,random_coefficient,random_coefficient_bias):
    expected_labels = np.matmul(input,random_coefficient) + random_coefficient_bias
    for i in range(expected_labels.shape[0]):
        if(expected_labels[i] >= 0):
            expected_labels[i] = 1
        else:
            expected_labels[i]= -1
    return expected_labels

In [426]:
coefficient,coeff_bias = implementRandomMethod(train_info,train_label,5)

In [427]:
test_random_expected = implementRandomMethodPredict(test_info,coefficient,coefficient_bias)

In [428]:
test_random_expected.shape

(6000, 1)

In [429]:
def implementPerceptronMethod(input,label,iterations,lr):
    perceptron_coefficient = np.zeros((1,100))
    perceptron_coefficient_bias = np.zeros(1)
    for t in range(iterations):
        for i in range(input.shape[0]):
            print(perceptron_coefficient)
            expected_labels = np.matmul(input[i].reshape(1,100),perceptron_coefficient.T) + perceptron_coefficient_bias
            if(expected_labels >= 0):
                expected_labels = 1
            else:
                expected_labels = -1
            error = label[i] - expected_labels
            print(error)
            perceptron_coefficient = perceptron_coefficient + lr*error*(input[i].reshape(1,100))
            print(perceptron_coefficient)
            perceptron_coefficient_bias = perceptron_coefficient_bias + lr*error
    return perceptron_coefficient.reshape(1,-1),perceptron_coefficient_bias

In [430]:
def implementPerceptronMethod(input,label,iterations):
    perceptron_coefficient = np.zeros((1,100))
    perceptron_coefficient_bias = np.zeros(1)
    for t in range(iterations):
        for i in range(input.shape[0]):
            expected_labels = np.matmul(input[i].reshape(1,100),perceptron_coefficient.T) + perceptron_coefficient_bias
            if(expected_labels*label[i] <= 0):
                perceptron_coefficient+=(label[i]*input[i].reshape(1,100))
                perceptron_coefficient_bias += label[i]
      
    return perceptron_coefficient.reshape(1,-1),perceptron_coefficient_bias

In [431]:
perceptron_coefficient,perceptron_bias = implementPerceptronMethod(train_info,train_label,20)

In [432]:
def implementPerceptronMethodPredict(input,perceptron_coeff,percept_bias):
    expected_labels = np.matmul(input,perceptron_coeff.T) + percept_bias
    for i in range(expected_labels.shape[0]):
        if(expected_labels[i] >= 0):
            expected_labels[i] = 1
        else:
            expected_labels[i]= -1
        return expected_labels.reshape(-1,1)

In [433]:
test_perceptronmethod_predict = implementPerceptronMethodPredict(test_info,perceptron_coefficient,perceptron_bias)

In [434]:
# YOUR CODE HERE
def implementGradientDescentMethod(input,label,iterations,learning_rate):
  teta = np.random.rand(100 + 1, 1)
  test_sampleData = input.shape[0]

  for t in range(iterations):
    expected = np.matmul(input, teta[1:].reshape(-1,1)) + teta[0]
    errors = expected - label
    # Calculate gradients
    gradient = np.matmul(input.T, errors) / test_sampleData

    # Update model parameters using gradient descent
    teta[1:] -= learning_rate * gradient
    teta[0] -= learning_rate * np.sum(errors) / test_sampleData
    # print(perceptron_coeff)
  return teta[1:].reshape(1,-1),teta[0]

In [435]:
gradient_descent_coefficient,gradient_descent_coefficient_bias = implementGradientDescentMethod(train_info,train_label,50,1e-2)

In [436]:
gradient_descent_coefficient

array([[2.81048085e+42, 2.84984119e+42, 2.88395203e+42, 2.88078913e+42,
        2.89037585e+42, 2.89680952e+42, 2.84874224e+42, 2.81128671e+42,
        2.87766544e+42, 2.81671019e+42, 2.85256329e+42, 2.86785163e+42,
        2.81324895e+42, 2.83417580e+42, 2.83659265e+42, 2.82952275e+42,
        2.87703973e+42, 2.83140170e+42, 2.89792330e+42, 2.85394268e+42,
        2.83057819e+42, 2.83158080e+42, 2.84425512e+42, 2.84625231e+42,
        2.85468515e+42, 2.82410660e+42, 2.88717663e+42, 2.85470582e+42,
        2.79517360e+42, 2.82170695e+42, 2.86594776e+42, 2.85606464e+42,
        2.85429323e+42, 2.85409931e+42, 2.84126246e+42, 2.83848835e+42,
        2.86633834e+42, 2.83743032e+42, 2.84624059e+42, 2.85540254e+42,
        2.87988269e+42, 2.88331185e+42, 2.84896256e+42, 2.87277128e+42,
        2.88448657e+42, 2.85975155e+42, 2.87349702e+42, 2.79065986e+42,
        2.82017598e+42, 2.88728357e+42, 2.84290299e+42, 2.83672367e+42,
        2.83537585e+42, 2.85683535e+42, 2.87240371e+42, 2.811876

In [437]:
gradient_descent_coefficient_bias

array([6.95225124e+41])

In [438]:
def implementGradientDescentMethodPredict(input,grad_coefficient,grad_bias):
    expected_labels = np.matmul(input,grad_coefficient.T) + grad_bias
    for i in range(expected_labels.shape[0]):
        if(expected_labels[i] >= 0):
            expected_labels[i] = 1
        else:
            expected_labels[i]= -1
    return expected_labels.reshape(-1,1)

In [439]:
test_gradientMethod_prediction = implementGradientDescentMethodPredict(test_info,gradient_descent_coefficient,gradient_descent_coefficient_bias)

### 4. Performance Evaluation
- Evaluate the performance of each method on the test set using various evaluation metrics discussed in the class, such as accuracy, precision, recall, F1-score. (You are expected to write the code from scratch for each of the metrics.)

In [440]:
# YOUR CODE HERE
def accuracyMethod(valid_labels,expected_labels):
    accurate_expectations = 0
    total_expectations = len(valid_labels)
    for valid_label, expected_label in zip(valid_labels, expected_labels):
        if valid_label == expected_label:
            accurate_expectations += 1
    accuracy_measurements = (accurate_expectations / total_expectations ) * 100
    return accuracy_measurements

def precisionMethod(valid_labels,expected_labels):
    accurate_positives = 0
    inaccurate_positives = 0
    for valid_label,expected_label  in zip(valid_labels,expected_labels):
        if valid_label == 1 and expected_label  == 1:
            accurate_positives += 1
        elif valid_label == -1 and expected_label == 1:
             inaccurate_positives += 1
    precision_measurements = accurate_positives / (accurate_positives + inaccurate_positives)
    return precision_measurements

def recallMethod(valid_labels,expected_labels):
    accurate_positives = 0
    inaccurate_positives = 0
    for valid_label,expected_label  in zip(valid_labels,expected_labels):
         if valid_label == 1 and expected_label  == 1:
            accurate_positives += 1
         elif valid_label == 1 and expected_label == -1:
             inaccurate_positives += 1
    recall_measurements = accurate_positives / (accurate_positives + inaccurate_positives)
    return recall_measurements

def F1_score(valid_labels,expected_labels):
  precision = precisionMethod(valid_labels,expected_labels)
  recall = recallMethod(valid_labels,expected_labels)
  f1_score = 2 * (precision * recall) / (precision + recall)
  return f1_score

In [441]:
# Random Method
print(f"Accuracy is : {accuracyMethod(test_random_expected,test_label)}")
print(f"Precision is : {precisionMethod(test_random_expected,test_label)}")
print(f"Recall is : {recallMethod(test_random_expected,test_label)}")
print(f"F1 Score is : {F1_score(test_random_expected,test_label)}")

Accuracy is : 25.650000000000002
Precision is : 0.511295681063123
Recall is : 0.33981011260763966
F1 Score is : 0.4082769598089932


In [442]:
print(f"Accuracy is : {accuracyMethod(test_perceptronmethod_predict,test_label)}")
print(f"Precision is : {precisionMethod(test_perceptronmethod_predict,test_label)}")
print(f"Recall is : {recallMethod(test_perceptronmethod_predict,test_label)}")
print(f"F1 Score is : {F1_score(test_perceptronmethod_predict,test_label)}")

Accuracy is : 0.016666666666666666
Precision is : 1.0
Recall is : 1.0
F1 Score is : 1.0


In [443]:
# Gradient Descent Method
print(f"Accuracy is : {accuracyMethod(test_gradientMethod_prediction,test_label)}")
print(f"Precision is : {precisionMethod(test_gradientMethod_prediction,test_label)}")
print(f"Recall is : {recallMethod(test_gradientMethod_prediction,test_label)}")
print(f"F1 Score is : {F1_score(test_gradientMethod_prediction,test_label)}")

Accuracy is : 26.35
Precision is : 0.5252491694352159
Recall is : 0.34587617589148983
F1 Score is : 0.41709537000395724


### 5. Determine the best method
- Analyze the evaluation results to determine which method is the best fit for solving this problem based on the chosen evaluation metrics. (Write your reasoning)

In [ ]:
# YOUR CODE HERE
F1 score determines the classifier effectiveness.since,perceptron has hightest F1 score which is 1. Therefore, it is the best method.

> By following these steps, you can systematically generate, split, implement, and evaluate the performance of different linear classification methods to identify the most suitable one for your specific problem.

## Extra Credits
- Implement cross validation on the best method and show the results. (Refer the lecture slides for more information)

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()